In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

%matplotlib notebook
%config IPCompleter.greedy=True

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from services.StockService import StockService
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta

print("GPU Available: ", tf.test.is_gpu_available())

from tensorflow.python.client import device_lib 
 print(device_lib.list_local_devices())

sns.set()

GPU Available:  False


In [3]:
df = pd.read_csv('../dataset/GOOG-year.csv')
# date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

print(df.head())


dfNew = StockService.get_stock_from_datestrings('hd', '2017-01-01', '2018-01-01')
print("Date col loc: " + str(dfNew.columns.get_loc("Date")))
date_ori = pd.to_datetime(dfNew.iloc[:, dfNew.columns.get_loc("Date") - 1]).tolist()

print(dfNew.head())

print("Done loading data.")

         Date        Open        High         Low       Close   Adj Close  \
0  2016-11-02  778.200012  781.650024  763.450012  768.700012  768.700012   
1  2016-11-03  767.250000  769.950012  759.030029  762.130005  762.130005   
2  2016-11-04  750.659973  770.359985  750.560974  762.020020  762.020020   
3  2016-11-07  774.500000  785.190002  772.549988  782.520020  782.520020   
4  2016-11-08  783.400024  795.632996  780.190002  790.510010  790.510010   

    Volume  
0  1872400  
1  1943200  
2  2134800  
3  1585100  
4  1350800  
Date col loc: 1
   Unnamed: 0        Date    Open    High     Low   Close  Adj Close  \
0     2376204  2017-12-29  190.74  190.74  189.53  189.53     189.53   
1     2382625  2017-12-28  190.91  190.98  189.64  189.78     189.78   
2     2389049  2017-12-27  190.60  191.49  190.01  190.19     190.19   
3     2395475  2017-12-26  188.53  190.42  188.34  190.36     190.36   
4     2401901  2017-12-22  188.20  188.46  187.27  188.13     188.13   

      Volu

In [4]:
minmax = MinMaxScaler().fit(df.iloc[:, 1:].astype('float32'))
df_log = minmax.transform(df.iloc[:, 1:].astype('float32'))
df_log = pd.DataFrame(df_log)
df_log.head()

,0,1,2,3,4,5
0,0.123285,0.093923,0.124080,0.112708,0.112708,0.280549
1,0.083119,0.054180,0.108808,0.090008,0.090008,0.296007
2,0.022265,0.055572,0.079545,0.089628,0.089628,0.337838
3,0.109713,0.105948,0.155523,0.160459,0.160459,0.217824
4,0.142359,0.141421,0.181922,0.188066,0.188066,0.166670


In [5]:
class Model:
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        def lstm_cell(size_layer):
            return tf.nn.rnn_cell.LSTMCell(size_layer, state_is_tuple = False)
#             return tf.keras.layers.LSTMCell(size_layer, state_is_tuple = False)
        
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell(
            [lstm_cell(size_layer) for _ in range(num_layers)],
            state_is_tuple = False,
        )
        self.X = tf.placeholder(tf.float32, (None, None, size))
        self.Y = tf.placeholder(tf.float32, (None, output_size))
        drop = tf.contrib.rnn.DropoutWrapper(
            rnn_cells, output_keep_prob = forget_bias
        )
        self.hidden_layer = tf.placeholder(
            tf.float32, (None, num_layers * 2 * size_layer)
        )
        self.outputs, self.last_state = tf.nn.dynamic_rnn(
            drop, self.X, initial_state = self.hidden_layer, dtype = tf.float32
        )
        self.logits = tf.layers.dense(self.outputs[-1], output_size)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(
            self.cost
        )

In [6]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 500
dropout_rate = 0.7
future_day = 50

In [7]:
tf.reset_default_graph()
modelnn = Model(
    0.01, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout_rate
)

default_session = tf.get_default_session()

if default_session is not None:
    default_session.close()

config = tf.ConfigProto()
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

W0718 21:45:34.571189  9032 deprecation.py:323] From <ipython-input-5-d0af02e86c66>:12: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0718 21:45:34.588137  9032 rnn_cell_impl.py:893] <tensorflow.python.ops.rnn_cell_impl.LSTMCell object at 0x000001C20B805FD0>: Using a concatenated state is slower and will soon be deprecated.  Use state_is_tuple=True.
W0718 21:45:34.590205  9032 deprecation.py:323] From <ipython-input-5-d0af02e86c66>:17: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0718 21:45:35.461835  9032 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in Te

In [8]:
num_records = df_log.shape[0]
for i in range(epoch):
    init_value = np.zeros((1, num_layers * 2 * size_layer))
    total_loss = 0
    for k in range(0, num_records - 1, timestamp):
        
        # Don't go higher than num_records
        index = min(k + timestamp, num_records - 1)
        batch_x = np.expand_dims(
            df_log.iloc[k : index, :].values, axis = 0
        )
        
        # Get the next rows + 5 more rows and all 6 columns
        batch_y = df_log.iloc[k + 1 : index + 1, :].values
      
        last_state, _, loss = sess.run(
            [modelnn.last_state, modelnn.optimizer, modelnn.cost],
            feed_dict = {
                modelnn.X: batch_x,
                modelnn.Y: batch_y,
                modelnn.hidden_layer: init_value,
            },
        )
        init_value = last_state
        total_loss += loss
    total_loss /= df_log.shape[0] // timestamp
    if (i + 1) % 100 == 0:
        print('epoch:', i + 1, 'avg loss:', total_loss)
print("done.")

epoch: 100 avg loss: 0.005231089351000264
epoch: 200 avg loss: 0.005782735673710704
epoch: 300 avg loss: 0.005615398443769664
epoch: 400 avg loss: 0.003964831046760082
epoch: 500 avg loss: 0.004420458514941856
done.


In [9]:
num_columns = df_log.shape[1]
output_predict = np.zeros((num_records + future_day, num_columns))
output_predict[0] = df_log.iloc[0]
upper_b = (num_records // timestamp) * timestamp
init_value = np.zeros((1, num_layers * 2 * size_layer))
for k in range(0, (num_records // timestamp) * timestamp, timestamp):
# for k in range(0, upper_b, timestamp):
    out_logits, last_state = sess.run(
        [modelnn.logits, modelnn.last_state],
        feed_dict = {
            modelnn.X: np.expand_dims(
                df_log.iloc[k : k + timestamp], axis = 0
            ),
            modelnn.hidden_layer: init_value,
        },
    )
    init_value = last_state
    output_predict[k + 1 : k + timestamp + 1] = out_logits

out_logits, last_state = sess.run(
    [modelnn.logits, modelnn.last_state],
    feed_dict = {
        modelnn.X: np.expand_dims(df_log.iloc[upper_b:], axis = 0),
        modelnn.hidden_layer: init_value,
    },
)
init_value = last_state
output_predict[upper_b + 1 : num_records + 1] = out_logits
df_log.loc[num_records] = out_logits[-1]
date_ori.append(date_ori[-1] + timedelta(days = 1))

In [ ]:
for i in range(future_day - 1):
    out_logits, last_state = sess.run(
        [modelnn.logits, modelnn.last_state],
        feed_dict = {
            modelnn.X: np.expand_dims(df_log.iloc[-timestamp:], axis = 0),
            modelnn.hidden_layer: init_value,
        },
    )
    init_value = last_state
    output_predict[df_log.shape[0]] = out_logits[-1]
    df_log.loc[df_log.shape[0]] = out_logits[-1]
    date_ori.append(date_ori[-1] + timedelta(days = 1))

In [ ]:
df_log = minmax.inverse_transform(output_predict)
date_ori = pd.Series(date_ori).dt.strftime(date_format = '%Y-%m-%d').tolist()

In [ ]:
def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [ ]:
current_palette = sns.color_palette('Paired', 12)
fig = plt.figure(figsize = (11, 10))
ax = plt.subplot(111)
x_range_original = np.arange(df.shape[0])
x_range_future = np.arange(df_log.shape[0])
# ax.plot(
#     x_range_original,
#     df.iloc[:, 1],
#     label = 'true Open',
#     color = current_palette[0],
# )
# ax.plot(
#     x_range_future,
#     anchor(df_log[:, 0], 0.5),
#     label = 'predict Open',
#     color = current_palette[1],
# )
ax.plot(
    x_range_original,
    df.iloc[:, 2],
    label = 'true High',
    color = current_palette[2],
)
ax.plot(
    x_range_future,
    anchor(df_log[:, 1], 0.5),
    label = 'predict High',
    color = current_palette[3],
)
# ax.plot(
#     x_range_original,
#     df.iloc[:, 3],
#     label = 'true Low',
#     color = current_palette[4],
# )
# ax.plot(
#     x_range_future,
#     anchor(df_log[:, 2], 0.5),
#     label = 'predict Low',
#     color = current_palette[5],
# )
# ax.plot(
#     x_range_original,
#     df.iloc[:, 4],
#     label = 'true Close',
#     color = current_palette[6],
# )
# ax.plot(
#     x_range_future,
#     anchor(df_log[:, 3], 0.5),
#     label = 'predict Close',
#     color = current_palette[7],
# )
# ax.plot(
#     x_range_original,
#     df.iloc[:, 5],
#     label = 'true Adj Close',
#     color = current_palette[8],
# )
# ax.plot(
#     x_range_future,
#     anchor(df_log[:, 4], 0.5),
#     label = 'predict Adj Close',
#     color = current_palette[9],
# )
box = ax.get_position()
ax.set_position(
    [box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9]
)
ax.legend(
    loc = 'upper center',
    bbox_to_anchor = (0.5, -0.05),
    fancybox = True,
    shadow = True,
    ncol = 5,
)
plt.title('overlap stock market')
plt.xticks(x_range_future[::30], date_ori[::30])
plt.rc('xtick',labelsize=6)
plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 8))
plt.subplot(1, 2, 1)
plt.plot(
    x_range_original,
    df.iloc[:, 1],
    label = 'true Open',
    color = current_palette[0],
)
plt.plot(
    x_range_original,
    df.iloc[:, 2],
    label = 'true High',
    color = current_palette[2],
)
plt.plot(
    x_range_original,
    df.iloc[:, 3],
    label = 'true Low',
    color = current_palette[4],
)
plt.plot(
    x_range_original,
    df.iloc[:, 4],
    label = 'true Close',
    color = current_palette[6],
)
plt.plot(
    x_range_original,
    df.iloc[:, 5],
    label = 'true Adj Close',
    color = current_palette[8],
)
plt.xticks(x_range_original[::60], df.iloc[:, 0].tolist()[::60])
plt.legend()
plt.title('true market')
plt.subplot(1, 2, 2)
plt.plot(
    x_range_future,
    anchor(df_log[:, 0], 0.5),
    label = 'predict Open',
    color = current_palette[1],
)
plt.plot(
    x_range_future,
    anchor(df_log[:, 1], 0.5),
    label = 'predict High',
    color = current_palette[3],
)
plt.plot(
    x_range_future,
    anchor(df_log[:, 2], 0.5),
    label = 'predict Low',
    color = current_palette[5],
)
plt.plot(
    x_range_future,
    anchor(df_log[:, 3], 0.5),
    label = 'predict Close',
    color = current_palette[7],
)
plt.plot(
    x_range_future,
    anchor(df_log[:, 4], 0.5),
    label = 'predict Adj Close',
    color = current_palette[9],
)
plt.xticks(x_range_future[::60], date_ori[::60])
plt.legend()
plt.title('predict market')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = plt.subplot(111)
ax.plot(x_range_original, df.iloc[:, -1], label = 'true Volume')
ax.plot(x_range_future, anchor(df_log[:, -1], 0.5), label = 'predict Volume')
box = ax.get_position()
ax.set_position(
    [box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9]
)
ax.legend(
    loc = 'upper center',
    bbox_to_anchor = (0.5, -0.05),
    fancybox = True,
    shadow = True,
    ncol = 5,
)
plt.xticks(x_range_future[::30], date_ori[::30])
plt.title('overlap market volume')
plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 8))
plt.subplot(1, 2, 1)
plt.plot(x_range_original, df.iloc[:, -1], label = 'true Volume')
plt.xticks(x_range_original[::60], df.iloc[:, 0].tolist()[::60])
plt.legend()
plt.title('true market volume')
plt.subplot(1, 2, 2)
plt.plot(x_range_future, anchor(df_log[:, -1], 0.5), label = 'predict Volume')
plt.xticks(x_range_future[::60], date_ori[::60])
plt.legend()
plt.title('predict market volume')
plt.show()